## Introduction
Ce rapport présente une analyse des risques détectés sur le lieu de travail à partir de données multi-modales (images, détections, météo, réglementation).

Ce fichier servira pour l’étape de préparation des données et l’étape d’entraînement, principalement en phase de préproduction. Une structure différente sera ensuite utilisée pour les besoins de la production.

### Données utilisées

    - Images panoramiques HD
    - Détections de personnes
    - Indications de risques
    - Météo

Voici un exemple de l’image ainsi qu’une description JSON des informations liées à cette image:
![Alt text](../assets/images_EST-1/654756511_999f6d8b-1331-4779-84e9-36b74bd21441.jpg "a sample image")

Extrait d'information json:
```
"654756511_999f6d8b-1331-4779-84e9-36b74bd21441.jpg": {
      "photo_id": 654756511,
      "photoset_id": 1374225,
      "image_shooting": "2025:06:16 05:50:07",
      "stitch_from": "stitching-tests/16db54d2-4541-4c48-8f34-68cef563cf06/3093724140/stitched/999f6d8b-1331-4779-84e9-36b74bd21441.jpg",
      "detections": [
        {
          "score": 0.1710205078125,
          "label": "person",
          "bounding_box_start_x": 0.9674935340881348,
          "bounding_box_end_x": 0.9687273502349854,
          "bounding_box_start_y": 0.6455875039100647,
          "bounding_box_end_y": 0.6539404988288879,
          "attributes": {
            "has_high_vis_pants": 0.0,
            "has_hard_hat": 0.0,
            "has_high_vis_vest": 0.0,
            "no_ppe": 0.0,
            "two_or_more": 0.0
          }
        },
        {
          "score": 0.25830078125,
          "label": "bird",
          "bounding_box_start_x": 0.5115770101547241,
          "bounding_box_end_x": 0.5254966020584106,
          "bounding_box_start_y": 0.6345036625862122,
          "bounding_box_end_y": 0.6598840355873108
        },...}
```

L’idée de ce projet est de développer un modèle LLM pour la gestion des risques. Le modèle prendra en entrée une image d’un chantier (ou d’un environnement similaire), une liste d’objets détectés (principalement des personnes) à partir de capteurs préexistants, ainsi que des informations météorologiques. Le modèle générera ensuite un rapport sur les meilleures pratiques à adopter pour atténuer les risques.

D’après les données, nous pouvons constater que pour chaque personne détectée, nous disposons d’un bounding box ainsi que de scores pour quatre critères de gestion des risques recherchés : port du casque, port du pantalon de sécurité, port du gilet de sécurité, et absence totale d’équipements de protection individuelle (PPE).

## Préparation des données
### La première étape:
Consiste à préparer les données pour l’entraînement ou le fine-tuning. Je vais commencer par mixer les deux dossiers d’images ainsi que les deux fichiers JSON. Je ne conserverai que les informations concernant les personnes ayant un score de détection d’objet supérieur ou égal à 0,5 :

1) Pour réduire le nombre de requêtes envoyées à l’API d’OpenAI.
2) L’étape de détection est considérée comme déjà entraînée et fine-tunée avant la génération du fichier JSON.

Le nouveau fichier json **images_EST_GT.json** va servir comme 'Ground Truth' pour l’entraînement.

In [1]:
# Libraries que nous allons utiliser
import os
import json
import time
import shutil
from PIL import Image
from openai import OpenAI
from dotenv import load_dotenv
from sklearn.metrics import accuracy_score
from utils.util import convert_image_to_base64, split_image, parse_llm_response
from utils.util import get_predictions, get_ground_truth

load_dotenv()

True

In [21]:
with open('../assets/images_EST-1.json', 'r', encoding='utf-8') as file:
    images_data_1 = json.load(file)['images']
with open('../assets/images_EST-2.json', 'r', encoding='utf-8') as file:
    images_data_2 = json.load(file)['images']

In [ ]:
accepted_score = 0.5
ground_truth = dict()

# Pour chaque image, nous allons vérifier les détections d'objets pour les personnes détectées.

meta_data = {'has_high_vis_pant': 0, 'has_hard_hat': 0, 'has_high_vis_vest': 0, 'no_ppe': 0}
for image_path, image_data in images_data_1.items():
    detections = []
    for detected_obj in image_data.get('detections', []):
        if detected_obj['score'] >= accepted_score and detected_obj['label'] == 'person':
            detections.append(detected_obj)
            for key, val in detected_obj.get('attributes', {}).items():
                if key in meta_data and val >= accepted_score:
                    meta_data[key] += 1

    if len(detections) > 0:
        ground_truth[image_path] = {'detections': detections, 'timestamp':image_data['image_shooting']}
    

for image_path, image_data in images_data_2.items():
    detections = []
    for detected_obj in image_data.get('detections', []):
        if detected_obj['score'] >= accepted_score and detected_obj['label'] == 'person':
            detections.append(detected_obj)
            for key, val in detected_obj.get('attributes', {}).items():
                if key in meta_data and val >= accepted_score:
                    meta_data[key] += 1
    if len(detections) > 0:
        ground_truth[image_path] = {'detections': detections, 'timestamp':image_data['image_shooting']}

In [ ]:
# Sauvegarder le résultat dans un fichier JSON.
with open('../assets/images_EST_GT.json', 'w', encoding='utf-8') as file:
    json.dump({'images': ground_truth, 'meta-info': meta_data}, file)

In [ ]:
# Copier les images dans le dossier de Ground Truth
for image in ground_truth:
    try:
        shutil.copyfile('../assets/images_EST-1/' + image, '../assets/images_EST_GT/' + image)
    except Exception as _:
        shutil.copyfile('../assets/images_EST-2/' + image, '../assets/images_EST_GT/' + image)

In [ ]:
# Tri des images par timestamp pour une meilleure organisation et qui va aider lors de l'entraînement avec les données de la météo.

with open('../assets/images_EST_GT.json', 'r', encoding='utf-8') as file:
    ground_truth = json.load(file)
sorted_gt = dict(sorted(ground_truth['images'].items(), key=lambda item: item[1]['timestamp']))

with open('../assets/images_EST_GT_sorted.json', 'w', encoding='utf-8') as file:
    json.dump({'images': sorted_gt, 'meta-info': ground_truth['meta-info']}, file, indent=4, ensure_ascii=False)

### La deuxième étape : 
Consiste à diviser les données en ensembles d’entraînement, de validation et de test:

- Entraînement : nous pouvons utiliser différents paramètres pour le traitement des images avec les LLMs, par exemple : haute ou basse résolution, ajout d’une marge autour des bounding boxes, etc.
- Validation : tester différents prompts et évaluer lesquels donnent les meilleurs résultats
- Test : simuler l’arrivée de nouvelles images

In [ ]:
# Train Validation Test Split
accepted_score = 0.5
train_len = int(0.6 * len(sorted_gt))
valid_len = int(0.8 * len(sorted_gt))

train_images = dict()
valid_images = dict()
test_images = dict()

# Les données meta pour assurer que nous avons un bon équilibre entre les ensembles d'entraînement, de validation et de test.
train_meta_data = {'has_high_vis_pant': 0, 'has_hard_hat': 0, 'has_high_vis_vest': 0, 'no_ppe': 0}
valid_meta_data = {'has_high_vis_pant': 0, 'has_hard_hat': 0, 'has_high_vis_vest': 0, 'no_ppe': 0}
test_meta_data = {'has_high_vis_pant': 0, 'has_hard_hat': 0, 'has_high_vis_vest': 0, 'no_ppe': 0}

for ind, img_path in enumerate(sorted_gt):
    img_obj = sorted_gt[img_path]
    if ind < train_len:
        train_images[img_path] = img_obj
        for detected_obj in img_obj.get('detections', []):
            for key, val in detected_obj.get('attributes', {}).items():
                if key in train_meta_data and val >= accepted_score:
                    train_meta_data[key] += 1

    elif ind < valid_len:
        valid_images[img_path] = img_obj
        for detected_obj in img_obj.get('detections', []):
            for key, val in detected_obj.get('attributes', {}).items():
                if key in valid_meta_data and val >= accepted_score:
                    valid_meta_data[key] += 1
    else:
        test_images[img_path] = img_obj
        for detected_obj in img_obj.get('detections', []):
            for key, val in detected_obj.get('attributes', {}).items():
                if key in test_meta_data and val >= accepted_score:
                    test_meta_data[key] += 1

In [ ]:
# Sauvegarder les ensembles d'entraînement, de validation et de test dans des fichiers JSON séparés.
with open('../assets/images_EST_GT_train.json', 'w', encoding='utf-8') as file:
    json.dump({'images': train_images, 'meta-info': train_meta_data}, file, indent=4, ensure_ascii=False)

with open('../assets/images_EST_GT_valid.json', 'w', encoding='utf-8') as file:
    json.dump({'images': valid_images, 'meta-info': valid_meta_data}, file, indent=4, ensure_ascii=False)

with open('../assets/images_EST_GT_test.json', 'w', encoding='utf-8') as file:
    json.dump({'images': test_images, 'meta-info': test_meta_data}, file, indent=4, ensure_ascii=False)

Pour l’entraînement, les paramètres suivants seront utilisés :

- Résolution de l’image : {high, low}
- Bounding Box avec une marge de 0,05 de la taille d'image originale.
- Prompt: **promt_1** a été utilisé durant l’entraînement en utilisant XML comme sorti.

**NOTE** : Je n’ai pas utilisé beaucoup de paramètres afin de minimiser le nombre de requêtes envoyées à l’API d’OpenAI.

In [ ]:
prompt_1 = '''You are a construction safety inspection assistant.

Your task is to detect the presence of PPE (personal protective equipment) on a person in the image. 
For each item below, return a detection of either 1 (yes) or 0 (no), and a confidence score between 0.0 and 1.0.

You must respond **only** using the following XML format. Do not add explanations or extra text and finish always with </no-ppe></output>.

<output>
    <has-hard-hat>
        <detection>{1 or 0}</detection>
        <confidence>{0.0 - 1.0}</confidence>
    </has-hard-hat>
    <has-high-vis-pants>
        <detection>{1 or 0}</detection>
        <confidence>{0.0 - 1.0}</confidence>
    </has-high-vis-pants>
    <has-high-vis-vest>
        <detection>{1 or 0}</detection>
        <confidence>{0.0 - 1.0}</confidence>
    </has-high-vis-vest>
    <no-ppe>
        <detection>{1 or 0}</detection>
        <confidence>{0.0 - 1.0}</confidence>
    </no-ppe>
</output>'''


llm = OpenAI(api_key=os.getenv('MY_OPENAI_API_KEY'))

In [26]:
def analyse_image(image, promt, resolution='high'):

    response = llm.chat.completions.create(
        model='gpt-4o-mini',
        temperature=0,
        messages=[
            {'role': 'system', 'content': 'You are a construction safety expert.'},
            {
                'role': 'user',
                'content': [
                    {'type': 'text', 'text': promt},
                        {
                            'type': 'image_url',
                            'image_url': {
                                'url': f'data:image/jpeg;base64,{image}', 'detail': resolution
                            }
                        }
                    ]
                }
            ],
            max_tokens=600,
            )

    content = response.choices[0].message.content
    return content

Pour chaque personne, nous découperons l’image selon sa boîte englobante, puis nous la soumettrons au LLM afin qu’il détecte les équipements requis, avec :

- une prédiction binaire (1 ou 0),
- un score de confiance (entre 0 et 1),
- et, si le LLM échoue à générer une sortie satisfaisante, un compteur d’échecs sera incrémenté.

In [60]:
def fit(file_path, out_file, prompt, resolution='high'):
    with open(file_path, 'r', encoding='utf-8') as file:
        train_images = json.load(file)

    train_images = train_images['images']
    
    llm_fails = 0
    new_images = []
    for image_path, img_obj in train_images.items():
        predictions = []
        img = Image.open(os.path.join('../assets/images_EST_GT/', image_path))
        for det_ind, d_obj in enumerate(img_obj['detections']):
            print(image_path, det_ind, llm_fails)
            cropped_image = split_image(img, d_obj['bounding_box_start_x'], d_obj['bounding_box_end_x'],
                                        d_obj['bounding_box_start_y'], d_obj['bounding_box_end_y'])
            cropped_img_64 = convert_image_to_base64(cropped_image)
            try:
                img_info = analyse_image(cropped_img_64, prompt, resolution=resolution)
                pred = parse_llm_response(img_info)
                predictions.append(pred)
            except Exception as _:
                predictions.append('-1')
                llm_fails += 1
            time.sleep(3)  # To avoid hitting rate limits
        json_info = {
            'timestamp': img_obj['timestamp'],
            'image_path': image_path,
            'detections': predictions
        }
        new_images.append(json_info)

    with open(out_file, 'w', encoding='utf-8') as file:
        json.dump({'images': new_images, 'LLM Failure': llm_fails}, file)

In [ ]:
# Lancer l'entraînement pour les résolutions 'high' et 'low'
for resolution in ['high', 'low']:
    fit('../assets/images_EST_GT_train.json', 
        '../assets/image_predictions_train_' + resolution + '.json', 
        resolution=resolution)
    print(f'Training completed for resolution: {resolution}')

In [5]:
with open('../assets/images_EST_GT_train.json', 'r', encoding='utf-8') as file:
    train_images = json.load(file)
with open('../assets/image_predictions_train_high.json', 'r', encoding='utf-8') as file:
    train_images_high = json.load(file)
with open('../assets/image_predictions_train_low.json', 'r', encoding='utf-8') as file:
    train_images_low = json.load(file)

In [8]:
has_hat_true = []
has_hat_high = []
has_hat_low = []

no_ppe_true = []
no_ppe_high = []
no_ppe_low = []


ground_truth = get_ground_truth(train_images)
has_hat_true, no_ppe_true = ground_truth['has_hat'], ground_truth['no_ppe']
high_predictions = get_predictions(train_images_high)
has_hat_high, no_ppe_high = high_predictions['has_hat'], high_predictions['no_ppe']
low_predictions = get_predictions(train_images_low)
has_hat_low, no_ppe_low = low_predictions['has_hat'], low_predictions['no_ppe']

In [ ]:
print('Accuracy HAT_HIGH', round(accuracy_score(has_hat_true, has_hat_high), 3))
print('Accuracy HAT_LOW', round(accuracy_score(has_hat_true, has_hat_low), 3))
print('Accuracy PPE_HIGH', round(accuracy_score(no_ppe_true, no_ppe_high), 3))
print('Accuracy PPE_LOW', round(accuracy_score(no_ppe_true, no_ppe_low), 3))

print('LLM Fails HIGH:', train_images_high['LLM Failure'])
print('LLM Fails LOW:', train_images_low['LLM Failure'])

HAT_HIGH 0.878
HAT_LOW 0.833
PPE_HIGH 0.878
PPE_LOW 0.833
LLM Fails HIGH: 4
LLM Fails LOW: 11


D’après les résultats précédents, l’utilisation d’images en haute résolution donne de meilleurs résultats. 

Ainsi, pour la validation des prompts, nous allons uniquement utiliser resolution="high".

In [49]:
# Une autre prompt pour la validation des prédictions LLM avec plus de détails et d'etre plus explicite sur les attentes.

prompt_2 = '''You are an AI safety inspector. You are shown a construction site image.

            Your only task is to check if the person in the image is wearing any of the following PPE (personal protective equipment):

            - A hard hat
            - A high-visibility vest
            - High-visibility pants
            - No PPE at all

            Only include items in the output if the model's confidence is greater than 0.5.  
            If confidence is ≤ 0.5, set `<prediction>` to 0 and still include the actual confidence value.
            Predictions must be based only on visual evidence in the image. 
            Do not assume PPE is present unless it is clearly visible with confidence > 0.5.
            Please respond using **only** the following XML structure. Do not include any extra text, explanation, or summaries. 
            Return only the content starting with <output> and ending with </output>.

            <output>
                <has-hard-hat>
                    <prediction>{1 or 0}</prediction>
                    <confidence>{0.0 - 1.0}</confidence>
                </has-hard-hat>
                <has-high-vis-pants>
                    <prediction>{1 or 0}</prediction>
                    <confidence>{0.0 - 1.0}</confidence>
                </has-high-vis-pants>
                <has-high-vis-vest>
                    <prediction>{1 or 0}</prediction>
                    <confidence>{0.0 - 1.0}</confidence>
                </has-high-vis-vest>
                <no-ppe>
                    <prediction>{1 or 0}</prediction>
                    <confidence>{0.0 - 1.0}</confidence>
                </no-ppe>
            </output>'''

In [61]:
# Lancer l'entraînement pour les résolutions 'high' et 'low'
prompts = [prompt_1, prompt_2]
for ind, prompt in enumerate(prompts):
    fit('../assets/images_EST_GT_valid.json', 
        '../assets/image_predictions_valid_prompt_' + str(ind+1) + '.json', prompt=prompt,
        resolution=resolution)
    print(f'Validation completed for prompt: {ind+1}')

663679906_5f1ef703-d7d7-4a18-809c-23b38679e1dd.jpg 0 0
664003659_e0e08173-7be7-4c02-8f53-675979c55e1d.jpg 0 0
664225860_219a5555-e487-49ca-850b-5a6974218ef3.jpg 0 0
664242924_1ca16733-6c77-4ed7-8d7a-06b9fe00b15b.jpg 0 0
664495381_42ecdef9-0bc2-40d1-86a0-c5dcaba83d68.jpg 0 0
664993276_ee64f700-1113-4170-a6bf-a5aeffcd726c.jpg 0 0
665241409_6d476db8-b162-4cfc-a446-e3987bddfca2.jpg 0 0
665769639_99620405-fc20-4dfd-bfd3-09d89ae748fd.jpg 0 0
666071019_a10b59d1-b209-4265-81a6-a377dd518797.jpg 0 0
666272086_7f8f06b3-a487-4161-95a8-0996f28242bd.jpg 0 0
667412533_ffc12494-bdb0-4652-8722-cdf4a5e57ef1.jpg 0 0
667429221_2feb1954-8caf-4f7c-919c-42e074ede61d.jpg 0 0
667429221_2feb1954-8caf-4f7c-919c-42e074ede61d.jpg 1 0
667666878_267fbc68-04ef-471c-a091-bfadf291b157.jpg 0 0
668210461_f9792838-213a-41e5-9e2d-69be22648a01.jpg 0 0
668210461_f9792838-213a-41e5-9e2d-69be22648a01.jpg 1 0
668223583_d0b3f8e3-d103-4527-8308-afca6d6a69a0.jpg 0 0
668223583_d0b3f8e3-d103-4527-8308-afca6d6a69a0.jpg 1 0
668745738_

In [10]:
with open('../assets/images_EST_GT_valid.json', 'r', encoding='utf-8') as file:
    valid_images = json.load(file)
with open('../assets/image_predictions_valid_prompt_1.json', 'r', encoding='utf-8') as file:
    valid_images_prompt_1 = json.load(file)
with open('../assets/image_predictions_valid_prompt_2.json', 'r', encoding='utf-8') as file:
    valid_images_prompt_2 = json.load(file)

In [11]:
has_hat_true = []
has_hat_prompt_1 = []
has_hat_prompt_2 = []

no_ppe_true = []
no_ppe_prompt_1 = []
no_ppe_prompt_2 = []

ground_truth = get_ground_truth(valid_images)
has_hat_true, no_ppe_true = ground_truth['has_hat'], ground_truth['no_ppe']
high_predictions = get_predictions(valid_images_prompt_1)
has_hat_prompt_1, no_ppe_prompt_1 = high_predictions['has_hat'], high_predictions['no_ppe']
low_predictions = get_predictions(valid_images_prompt_2)
has_hat_prompt_2, no_ppe_prompt_2 = low_predictions['has_hat'], low_predictions['no_ppe']

In [ ]:
print('Accuracy HAT_PROMPT_1', round(accuracy_score(has_hat_true, has_hat_prompt_1), 3))
print('Accuracy HAT_PROMPT_2', round(accuracy_score(has_hat_true, has_hat_prompt_2), 3))
print('Accuracy PPE_PROMPT_1', round(accuracy_score(no_ppe_true, no_ppe_prompt_1), 3))
print('Accuracy PPE_PROMPT_2', round(accuracy_score(no_ppe_true, no_ppe_prompt_2), 3))

print('LLM Fails PROMPT_1:', valid_images_prompt_1['LLM Failure'])
print('LLM Fails PROMPT_2:', valid_images_prompt_2['LLM Failure'])

HAT_PROMPT_1 0.85
HAT_PROMPT_2 0.875
PPE_PROMPT_1 0.85
PPE_PROMPT_2 0.875
LLM Fails PROMPT_1: 2
LLM Fails PROMPT_2: 1


In [67]:
# Lancer l'entraînement pour les résolutions 'high' et 'low'
fit('../assets/images_EST_GT_test.json', 
    '../assets/image_predictions_test.json', prompt=prompt_2,
    resolution='high')
print(f'Testing completed')

670484413_30a9e6a7-1a81-4f83-89d2-d14f5093f4e1.jpg 0 0
670727037_f607ad68-c8bb-4b3f-a477-65ada65153f3.jpg 0 0
671945586_f6bfc504-4033-4c0f-b5ab-c1bed8b9d62e.jpg 0 0
671966647_270f53ad-9584-4ed9-9dfb-8e7b25da322d.jpg 0 0
671990197_90b445bd-97dd-4fea-b6b4-7fa17fbbf3fb.jpg 0 0
671999665_23d821e2-7359-41b1-9ae7-aa2fb43ef604.jpg 0 0
671999665_23d821e2-7359-41b1-9ae7-aa2fb43ef604.jpg 1 0
672014395_d583f064-4851-46f7-9276-c0fd345dafe0.jpg 0 0
672014395_d583f064-4851-46f7-9276-c0fd345dafe0.jpg 1 0
672014395_d583f064-4851-46f7-9276-c0fd345dafe0.jpg 2 0
672014347_2eeb042f-86c5-4724-a4a1-b674aa0be21c.jpg 0 0
672014347_2eeb042f-86c5-4724-a4a1-b674aa0be21c.jpg 1 0
672014347_2eeb042f-86c5-4724-a4a1-b674aa0be21c.jpg 2 0
672027488_bde0f884-99fb-4783-8183-243bc1801962.jpg 0 0
672045392_087ebdab-5484-4888-bd09-f9a15cc4339b.jpg 0 0
672045974_af43b6db-6224-4ee4-afaa-3ac5afa71e6a.jpg 0 0
672063142_3d1bcb4a-c78f-4df5-9d44-dbd4c0b64242.jpg 0 0
672092854_f52b37a0-a30c-4d60-a18c-4da2afdedc12.jpg 0 0
672092854_

In [13]:
with open('../assets/images_EST_GT_test.json', 'r', encoding='utf-8') as file:
    test_images = json.load(file)
with open('../assets/image_predictions_test.json', 'r', encoding='utf-8') as file:
    test_predictions = json.load(file)

In [ ]:
has_hat_true = []
has_hat_test = []

no_ppe_true = []
no_ppe_test = []

ground_truth = get_ground_truth(test_images)
has_hat_true, no_ppe_true = ground_truth['has_hat'], ground_truth['no_ppe']
predictions = get_predictions(test_predictions)
has_hat_test, no_ppe_test = predictions['has_hat'], predictions['no_ppe']

print('Accuracy HAT_TEST', round(accuracy_score(has_hat_true, has_hat_test), 3))
print('Accuracy PPE_TEST', round(accuracy_score(no_ppe_true, no_ppe_test), 3))

print('LLM Fails TEST:', test_predictions['LLM Failure'])

Accuracy HAT_TEST 0.81
Accuracy PPE_TEST 0.786
LLM Fails TEST: 1
